In [ ]:
import sys
sys.path.append("..")

In [ ]:
import numpy as np
import xarray as xr
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
from maps import map_robinson
from GMST import GMST_timeseries, GMST_regression
from paths import file_ex_atm_T_T850_U_V_yrly, path_results, path_samoc
from plotting import shifted_color_map
from constants import abs_zero
from timeseries import IterateOutputCESM
from xr_integrate import xr_surf_mean, xr_zonal_mean
from xr_DataArrays import xr_AREA

In [ ]:
AREA = xr_AREA('atm')

In [ ]:
ds = xr.open_dataset(file_ex_atm_T_T850_U_V_yrly, decode_times=False)
print(f'The GMST is {xr_surf_mean(ds["T"][-1,:,:], AREA=AREA)+abs_zero:5.2f} deg C.')

# GMST overview

In [ ]:
%%time
gmst_ctrl = GMST_timeseries('ctrl')
gmst_rcp  = GMST_timeseries('rcp')

In [ ]:
gmst_ctrl = xr.open_dataset(f'{path_results}/GMST/GMST_ctrl.nc')
gmst_rcp  = xr.open_dataset(f'{path_results}/GMST/GMST_rcp.nc' )

In [ ]:
gmst_rcp

In [ ]:
fig = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot(gmst_ctrl.year    , gmst_ctrl.GMST.values, lw=3, label='ctrl')
plt.plot(gmst_rcp.year-1800, gmst_rcp.GMST.values,  lw=3, label='RCP')
plt.xlabel('time [years]', fontsize=16)
plt.ylabel('GMST [$^\circ$C]', fontsize=16)
plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/GMST_rcp_ctrl')

In [ ]:
N_ctrl = len(gmst_ctrl.coords['year'])
N_rcp  = len(gmst_rcp.coords['year'])
print(N_ctrl, N_rcp)

fig = plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
for i in range(N_rcp):
    plt.plot(gmst_rcp.coords['lat'], 
             gmst_rcp.T_zonal[i,:]-gmst_ctrl.T_zonal[:,:].mean(dim='year'), 
             color=plt.cm.rainbow(i/N_rcp), alpha=.4, lw=.5,
             label='')
for i in range(N_rcp):
    if (i-5)%10==0:
        plt.plot(gmst_rcp.coords['lat'], 
                 gmst_rcp.T_zonal[i-5:i+5,:].mean(dim='year')-gmst_ctrl.T_zonal[:,:].mean(dim='year'), 
                 color=plt.cm.rainbow(i/N_rcp), alpha=1, lw=2.5, ls='--',
                 label=f'decade {int((i-5)/10)+1}')
plt.axhline(0, c='k', lw=.5)
plt.legend(ncol=3, frameon=False, fontsize=16)
plt.ylabel('zonally averaged temp. RCP - CTRL [K]', fontsize=16)
plt.xlabel('Latitude', fontsize=16)
plt.xticks(np.arange(-90,91,30))
plt.tight_layout()
plt.savefig(f'{path_results}/GMST/T_zonal_rcp-ctrl')

# Trends

In [ ]:
%%time
trends_ctrl = GMST_regression('ctrl')
trends_ctrl.to_netcdf(path=f'{path_results}/GMST/trend_ctrl.nc' , mode='w')
trends_rcp  = GMST_regression('rcp')
trends_rcp.to_netcdf(path=f'{path_results}/GMST/trend_rcp.nc' , mode='w')

In [ ]:
trends_ctrl = xr.open_dataarray(f'{path_results}/GMST/trend_ctrl.nc')
trends_rcp  = xr.open_dataarray(f'{path_results}/GMST/trend_rcp.nc' )

In [ ]:
for i, trends in enumerate([trends_ctrl, trends_rcp]):
    run = ['ctrl', 'rcp'][i]
    label = 'Surface temperature trend [K/century]'
    minv, maxv = -2, 6
    cmap = shifted_color_map(mpl.cm.RdBu_r, start=.33, midpoint=0.5, stop=1., name='shrunk')
    filename = f'{path_results}/GMST/T_trend_map_{run}'
    
    map_robinson(xa=trends, domain='atm', cmap=cmap, minv=minv, maxv=maxv, label=label, filename=filename)
    